# Lab 3

### Team Members:
 - Alex Lopez
 - Chris Haub
 - Erin McClure-Price
 - Chad Kwong

#### Library Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import turicreate as tc
import warnings


#### Data Import

In [2]:
%%time

#Loading all data file separately 
vle = pd.read_csv('vle.csv')
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
studentAssessments = pd.read_csv('studentAssessment.csv')
studentInfo = pd.read_csv('studentInfo.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')
studentVle = pd.read_csv('studentVle.csv')

Wall time: 4.44 s


In [3]:
print('studentVle: \n',studentVle.head())
print('studentVle shape: ',studentVle.shape,'\n\n')

print('assessments: \n',assessments.head())
print('assessments shape: ', assessments.shape, '\n\n')

print('courses: \n',courses.head())
print('courses shape', courses.shape, '\n\n')

print('studentAssessments: \n', studentAssessments.head())
print('studentAssessments shape: ', studentAssessments.shape,'\n\n')

print('studentInfo: \n', studentInfo.head())
print('studentInfo shape', studentInfo.shape,'\n\n')

print('studentRegistration: \n', studentRegistration.head())
print('studentRegistration shape: ', studentRegistration.shape,'\n\n')

print('vle: \n', vle.head())
print('vle shape: ', vle.shape,'\n\n')

studentVle: 
   code_module code_presentation  id_student  id_site  date  sum_click
0         AAA             2013J       28400   546652   -10          4
1         AAA             2013J       28400   546652   -10          1
2         AAA             2013J       28400   546652   -10          1
3         AAA             2013J       28400   546614   -10         11
4         AAA             2013J       28400   546714   -10          1
studentVle shape:  (10655280, 6) 


assessments: 
   code_module code_presentation  id_assessment assessment_type   date  weight
0         AAA             2013J           1752             TMA   19.0    10.0
1         AAA             2013J           1753             TMA   54.0    20.0
2         AAA             2013J           1754             TMA  117.0    20.0
3         AAA             2013J           1755             TMA  166.0    20.0
4         AAA             2013J           1756             TMA  215.0    30.0
assessments shape:  (206, 6) 


courses: 
   co

In [4]:
%%time
dfFull = pd.merge(studentVle, studentInfo, on=['code_module', 'code_presentation', 'id_student'])
print(dfFull.shape)
dfFull.head()

(10655280, 15)
Wall time: 4.86 s


,code_module,code_presentation,id_student,id_site,date,sum_click,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,28400,546652,-10,4,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
1,AAA,2013J,28400,546652,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,28400,546652,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
3,AAA,2013J,28400,546614,-10,11,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
4,AAA,2013J,28400,546714,-10,1,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass


# Business Understanding 1 #
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

# Data Understanding 1 #
*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

 - VLE stands for Virtual Learning Environment
 - VLE table records interactions with VLE 
 - 32,593 students
 - 22 courses
 

In [5]:
%%time
# Randomly sampling 30,000 records StudentVle Table with Million Rows.
studentVleSample = studentVle.sample(n=35000, random_state = 777)
print(studentVleSample.shape)

# vle
# assessments
# courses
# studentAssessments
# studentInfo
# studentRegistration
# studentVleSample

df = pd.merge(studentVleSample, studentInfo, on=['code_module', 'code_presentation', 'id_student'])
df2 = pd.merge(studentAssessments, assessments, on='id_assessment')
df = pd.merge(df, df2, on=['code_module', 'code_presentation', 'id_student'])
print(df.shape)
df.head()

(35000, 6)
(293018, 22)
Wall time: 1.12 s


,code_module,code_presentation,id_student,id_site,date_x,sum_click,gender,region,highest_education,imd_band,...,studied_credits,disability,final_result,id_assessment,date_submitted,is_banked,score,assessment_type,date_y,weight
0,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15020,17,0,100.0,TMA,19.0,0.0
1,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15021,51,0,70.0,TMA,54.0,10.0
2,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15022,107,0,72.0,TMA,110.0,20.0
3,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15023,152,0,60.0,TMA,152.0,35.0
4,BBB,2014J,584539,913685,17,1,F,Scotland,HE Qualification,10-20,...,60,N,Pass,15024,206,0,63.0,TMA,201.0,35.0


# Data Understanding 2 #
*Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

# Modeling and Evaluation 1 #
*Train and adjust parameters*

# Modeling and Evaluation 2	#
*Evaluate and Compare*

# Modeling and Evaluation 3 #
*Visualize Results*

# Modeling and Evaluation 4 #
*Summarize the Ramifications*

# Deployment #
*Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*

# Exceptional Work #